In [ ]:
pip install pycountry pandas regex requests

In [7]:
import requests
import zipfile
import os
import pycountry
import pandas as pd
import regex as re
from datetime import datetime

#The "download the temperature data" function
def DownloadFile(url, writeLocation):
    if requests.get(url).status_code == 200:
        with open(writeLocation, 'wb') as file:
            file.write(requests.get(url).content)
        return file
    else:
        return f"exited with {requests.get(url).status_code} as error"

def ExtractZip(zipFile, extractPath):
    try:
        with zipfile.ZipFile(zipFile, 'r') as zip_ref:
            zip_ref.extractall(extractPath)
        return extractPath
    except Exception as e:
        return f"Error: {e}"

def DownloadExtract(url, path):
    ExtractZip(DownloadFile(url, path), path)
    

def GenerateTemperatureMapping(folderPath, save=True):
    list = []
    STAIDCountry = {}
    reg = r"[\s]*(\d+),([\w\W\s]*),([a-zA-Z]{2})"
    EUList = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Denmark", "Estonia", 
              "Finland", "France", "Germany", "Ireland", "Italy", "Latvia", "Luxembourg",
              "Netherlands", "Norway", "Poland", "Romania", "Spain", "Sweden", "Switzerland", 
              "United Kingdom"]
    dataframeDict = {key: [] for key in EUList}
    i = 0

    
    #Lees het Satations.txt bestand,dit heeft de index van .txt bestanden->landen
    with open(folderPath + 'stations.txt') as f:
        lines = f.readlines()
        lines = lines[17:]
        f.close()
    
    #Genereer de dictionary die ID : land bijhoudt
    #TODO: Dictionary [land] : [lijst aan IDs] als STAIDCountry?
    for line in lines:
        m = re.match(reg, line)
        if m:
            if m.group(3) not in list:
                list.append(m.group(3))
            country = pycountry.countries.get(alpha_2=m.group(3)).name
            id = str(m.group(1)).zfill(5)
            STAIDCountry[id] = country
            
    # Voor elk .txt bestand, match de ID (gelezen door regex) met de key in de dictionary, en hernoem het bestand, zo genereert het LAND[ID].txt files 
    for file in os.listdir(folderPath):
        reg = r"TG_STAID0(\d*)"
        match = re.match(reg, file)
        if not match:
            continue
        id = match.group(1)
        newName = STAIDCountry[id] + id + '.txt'
        os.rename(os.path.join(folderPath, file), os.path.join(folderPath, newName))

    
    for txtFile in os.listdir(folderPath):
        tempList = []
        dateList = []
        start = False
        temp = []
        
        with open(folderPath + txtFile) as f:
            lines = f.readlines()
            f.close()
            
        for line in lines:
            reg = r"[\d]+,\s*\d+,(\d{8}),\s*([-\d]+),\s+(\d)"
            m = re.match(reg, line.strip())
            if m:
                if m.group(3) != "0" and m.group(3) != "1":
                    continue   
                # /10 omdat alle temperatuure in stappen van 0.1 graden celcius
                dateList.append(datetime.strptime(m.group(1), "%Y%m%d").date())
                tempList.append(int(m.group(2))/10)
            else:
                temp.append(line)
                
        df = pd.DataFrame({'Date': dateList, 'Temperature' : tempList})

        reg = r"([a-zA-Z\ ]+)(\d+)"
        m = re.match(reg, txtFile)
        if not m:
            print(reg)
            print(txtFile)
            continue
        country = m.group(1)
        if country in dataframeDict.keys():
            dataframeDict[country].append(df)
        i += 1
        print(i)
        if i>9:
            break
        
    if save:
        for country, dfList in dataframeDict.items():
            print(country, dfList)
            if dfList != []:
                pd.concat(dfList).to_csv("data/temperature/"+str(country)+".csv", index = False)
            #  pd.concat(dfList).to_csv("data/temperature/"+str(country), index = False)
            else:
                print(f"er zijn geen dataframes gevonden voor {country}")
    
    return dataframeDict


# temperatuur is in 0.1C, dus alles x 0.1 voordat het de database in geflikkerd wordt


print(GenerateTemperatureMapping('data/temp/'))

1
2
3
4
5
6
7
8
9
10
Austria []
Belgium []
Bulgaria []
Croatia []
Cyprus []
Denmark []
Estonia []
Finland []
France [             Date  Temperature
0      1923-01-01          2.4
1      1923-01-02          2.6
2      1923-01-03          5.2
3      1923-01-04          3.6
4      1923-01-05          2.4
...           ...          ...
36763  2023-08-27         18.6
36764  2023-08-28         16.8
36765  2023-08-29         17.2
36766  2023-08-30         18.5
36767  2023-08-31         19.3

[36768 rows x 2 columns]]
Germany [             Date  Temperature
0      1978-05-01          9.7
1      1978-05-02          7.5
2      1978-05-03          8.5
3      1978-05-04         11.7
4      1978-05-05         15.6
...           ...          ...
14273  2023-08-27         15.9
14274  2023-08-28         13.9
14275  2023-08-29         12.3
14276  2023-08-30         14.6
14277  2023-08-31         14.5

[14278 rows x 2 columns],              Date  Temperature
0      1940-01-01         -7.0
1      1940-01